# Wikipediaにあるページとその出身地を取得

In [1]:
from time import sleep
from collections import defaultdict
import json
from itertools import chain

from SPARQLWrapper import SPARQLWrapper
from tqdm.notebook import tqdm
import pandas as pd

from utils import PREFECTURES

In [2]:
sparql = SPARQLWrapper(endpoint="http://ja.dbpedia.org/sparql", returnFormat="json")

## 「<都道府県>出身の人物一覧」からのリンク先ページを列挙

e.g., [北海道出身の人物一覧 - Wikipedia](https://ja.wikipedia.org/wiki/%E5%8C%97%E6%B5%B7%E9%81%93%E5%87%BA%E8%BA%AB%E3%81%AE%E4%BA%BA%E7%89%A9%E4%B8%80%E8%A6%A7)

- 参考
  - [Template‐ノート:日本出身の人物の合意事項 - Wikipedia](https://ja.wikipedia.org/wiki/Template%E2%80%90%E3%83%8E%E3%83%BC%E3%83%88:%E6%97%A5%E6%9C%AC%E5%87%BA%E8%BA%AB%E3%81%AE%E4%BA%BA%E7%89%A9%E3%81%AE%E5%90%88%E6%84%8F%E4%BA%8B%E9%A0%85)
  - [出身別の人名記事一覧の一覧 - Wikipedia](https://ja.wikipedia.org/wiki/%E5%87%BA%E8%BA%AB%E5%88%A5%E3%81%AE%E4%BA%BA%E5%90%8D%E8%A8%98%E4%BA%8B%E4%B8%80%E8%A6%A7%E3%81%AE%E4%B8%80%E8%A6%A7#%E6%97%A5%E6%9C%AC%E3%81%AE%E5%9C%B0%E5%9F%9F%EF%BC%88%E9%83%BD%E9%81%93%E5%BA%9C%E7%9C%8C%E3%80%81%E5%B8%82%E7%94%BA%E6%9D%91%EF%BC%89)

In [3]:
pref_pages = defaultdict(set)

for pref_name in tqdm(PREFECTURES):
    sparql.setQuery(f"""
        SELECT DISTINCT * 
        WHERE {{
            <http://ja.dbpedia.org/resource/{pref_name}出身の人物一覧>
            dbo:wikiPageWikiLink
            ?o .
            }}
    """)
    res = sparql.query().convert()
    for x in res["results"]["bindings"]:
        assert x["o"]["type"] == "uri"
        pref_pages[pref_name].add( x["o"]["value"] )

  0%|          | 0/47 [00:00<?, ?it/s]

In [4]:
df_count = pd.DataFrame([(k, len(v)) for k, v in pref_pages.items()], columns=["pref", "n_pages"])
df_count

,pref,n_pages
0,北海道,3493
1,青森県,1428
2,岩手県,1313
3,宮城県,1574
4,秋田県,1002
5,山形県,1056
6,福島県,1991
7,茨城県,1401
8,栃木県,1133
9,群馬県,1649


In [5]:
all_page_list = sorted(set(chain.from_iterable(pref_pages.values())))
len(all_page_list)

72567

In [6]:
with open("../data/workspace/all_page_list.json", "w") as fp:
    json.dump(all_page_list, fp, indent=2, ensure_ascii=False)

In [7]:
!head ../data/workspace/all_page_list.json

[
  "http://ja.dbpedia.org/resource/!wagero!",
  "http://ja.dbpedia.org/resource/%22E%22qual",
  "http://ja.dbpedia.org/resource/%22brother%22YASSHI",
  "http://ja.dbpedia.org/resource/%22黒天使%22沼澤邪鬼",
  "http://ja.dbpedia.org/resource/%5C(%5Eo%5E)/チエ",
  "http://ja.dbpedia.org/resource/(劇団4ドル50セント)",
  "http://ja.dbpedia.org/resource/04_Limited_Sazabys",
  "http://ja.dbpedia.org/resource/0930",
  "http://ja.dbpedia.org/resource/1.FCケルン",


## 出身地を取得

In [8]:
def get_locations(page):
    sparql.setQuery(f"""
        SELECT DISTINCT * 
        WHERE {{
            <{page}>
            prop-ja:出身地 | prop-ja:出生地 | prop-ja:生誕地 | dbo:birthPlace | prop-ja:origin
            ?o .
            }}
    """)
    
    res = sparql.query().convert()
    items = res["results"]["bindings"]
    locations = set([x["o"]["value"] for x in items])
    return locations

In [9]:
get_locations("http://ja.dbpedia.org/resource/米津玄師")

{'・徳島県徳島市'}

In [10]:
get_locations("http://ja.dbpedia.org/resource/尾崎将司")

{'http://ja.dbpedia.org/resource/徳島県', '徳島県海部郡宍喰町（後の海陽町）', '徳島県海部郡宍喰町（現・海陽町）'}

In [11]:
get_locations("http://ja.dbpedia.org/resource/大谷翔平")

{'http://ja.dbpedia.org/resource/岩手県', '岩手県水沢市（現：奥州市）'}

### 出身地を取得できない例

あくまで構造化された情報として付与されたものを対象としている。

そのため、例えば自由記述で `○○はA県B市C区出身の人物である` などと記載されている**のみ**の場合は対象外となる

[About: 池田幸太郎 (首長)](https://ja.dbpedia.org/page/%E6%B1%A0%E7%94%B0%E5%B9%B8%E5%A4%AA%E9%83%8E_(%E9%A6%96%E9%95%B7))

> 池田幸太郎（いけだ こうたろう、1904年 - 1989年）は、日本の政治家・官僚・薬剤師。北海道旭川市出身。

しかし、プロパティとしてその情報（「北海道旭川市出身」）は付与されていないため、今回の手法では出身地なしとなる。

In [12]:
get_locations("http://ja.dbpedia.org/page/池田幸太郎_(首長)")

set()

### 全件取得

In [13]:
with open("../data/workspace/all_page_list.json") as fp:
    all_page_list = json.load(fp)
len(all_page_list)

72567

In [14]:
with open("../data/workspace/page_locations_raw.json") as fp:
    page_locations_raw = json.load(fp)

In [ ]:
for page in tqdm(all_page_list):
    if page in tqdem(page_locations_raw):
        continue
    
    sleep(1)
    locations = get_locations(page)
    if not locations:
        continue

    page_locations_raw[page] = sorted(locations)

In [ ]:
with open("../data/workspace/page_locations_raw.json", "w") as fp:
    json.dump(page_locations_raw, fp, indent=2, ensure_ascii=False)

In [15]:
!head ../data/workspace/page_locations_raw.json

{
  "%22E%22qual": [
    "愛知県名古屋市"
  ],
  "%22brother%22YASSHI": [
    "http://ja.dbpedia.org/resource/京都府",
    "京都府宇治市"
  ],
  "%22黒天使%22沼澤邪鬼": [
    "http://ja.dbpedia.org/resource/埼玉県",


In [16]:
!tail ../data/workspace/page_locations_raw.json

    "・栃木県佐野市"
  ],
  "［Alexandros］": [
    "・神奈川県相模原市",
    "愛知県日進市",
    "神奈川県",
    "神奈川県川崎市",
    "神奈川県相模原市"
  ]
}

### その他

In [17]:
# 出身地を紐付けられなかったページ
no_location_pages = [p for p in all_page_list if p.replace("http://ja.dbpedia.org/resource/", "") not in page_locations_raw]

with open("../data/workspace/no_location_pages.json", "w") as fp:
    json.dump(no_location_pages, fp, indent=2, ensure_ascii=False)

len(no_location_pages)

34828

In [18]:
# この段階では「人物」であるかは分からない点に注意（あくまで「出身地が付与されているページ」 ）
page_locations_raw["くるり"]

['京都府京都市']